In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random

In [12]:
def butuduomenys(kur, kiek):

    opcijos = Options()
    opcijos.add_argument('--incognito')
    driver = webdriver.Chrome(options=opcijos)

    url = f'https://www.aruodas.lt/butai/{kur}/puslapis/{kiek}/?FOwnerDbId0=1&FOwnerDbId2=1' 
    driver.get(url)
    time.sleep(1)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    aruodopirmas = bs.find_all('div', {'class':'advert-flex'})

    nuorodos=[]
    plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas = [], [], [], [], [], [], [], []
    energetineklase, ilguma, platuma, miestas, kaina, rajonas = [], [], [], [], [], []

    for aruodpirm in aruodopirmas:
        try:
            nuorodos.append(aruodpirm.find('a')['href'])
            url1 = nuorodos[-1] 
            driver.get(url1)
            time.sleep(1)
            source1 = driver.page_source
            bs1 = BeautifulSoup(source1, 'html.parser')
        except:
            pass
        else:
            asas=bs1.find('dl', {'class': 'obj-details'})
            # print(asas.prettify())

            bandymas=asas.find_all('dt')
            band=asas.find_all('dd')
            bandymas=[i.text.strip() for i in bandymas]
            band=[i.find('span', {'class': 'fieldValueContainer'}) for i in band]
            band2= [i.text.strip() if i is not None else str(None) for i in band]
            plotas.append(None)
            kambariusk.append(None)
            aukstas.append(None)
            aukstusk.append(None)
            metai.append(None)
            pastatotipas.append(None)
            sildymas.append(None)
            irengimas.append(None)
            energetineklase.append(None)
            ilguma.append(None)
            platuma.append(None)
            kaina.append(None)
            rajonas.append(None)
            
            if kur== 'vilniuje':
                miestas.append('Vilnius')
            elif kur== 'kaune':
                miestas.append('Kaunas')
            elif kur=='palangoje':
                miestas.append('Palanga')
            elif kur=='klaipedoje':
                miestas.append('Klaipeda')
            elif kur=='alytuje':
                miestas.append('Alytus')

            for p, k in zip(bandymas, band2):
                if p == 'Plotas:':
                    plotas[-1]=k
                if p == 'Kambarių sk.:':
                    kambariusk[-1]=k
                if p == 'Aukštas:':
                    aukstas[-1]=k
                if p == 'Aukštų sk.:':
                    aukstusk[-1]=k
                if p == 'Metai:':
                    metai[-1]=k
                if p == 'Pastato tipas:':
                    pastatotipas[-1]=k
                if p == 'Šildymas:':
                    sildymas[-1]=k
                if p == 'Įrengimas:':
                    irengimas[-1]=k
                if p == 'Pastato energijos suvartojimo klasė:':
                    energetineklase[-1]=k
                
            try:    
                tim=bs1.find('a', {'class':'link-obj-thumb vector-thumb-map'})
                xy=str(tim['href']).split('=')[-1].split('%2C')
            except:
                pass
            else:
                ilguma[-1]=xy[0]
                platuma[-1]=xy[1]
            try:
                apci= aruodpirm.find('span', {'class': 'list-item-price-v2'})
                kaina[-1]=((int(apci.text.strip().replace(' ', '').replace('€',''))))
            except:
                pass
            try:
                apci1= aruodpirm.find_all('img')
                rajonas[-1]=apci1[0]['title'].split(',')[0]
            except:
                pass
        
        # if aruodpirm == aruodopirmas[4]:
        #     break
    driver.close()
    return plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas, energetineklase, ilguma, platuma, miestas, kaina, rajonas
    

mm=['a']
kk=5
print('Įveskite miesto raidę ir kiek minučių veiks jūsų programa: \n v - Vilnius, ka - Kaunas, p - Palanga, kl - Klaipėda, a - Alytus  \n Parasykite kiek minučių veiks skriptas')

for i in range(0, len(mm)):
    if mm[i]== 'v':
        mm[i]='vilniuje'
    elif mm[i]== 'ka':
        mm[i]='kaune'
    elif mm[i]=='p':
        mm[i]='palangoje'
    elif mm[i]=='kl':
        mm[i]='klaipedoje'
    elif mm[i]=='a':
        mm[i]='alytuje'


for i in range(1, kk+1):
    for mst in mm:
        plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas, energetineklase, ilguma, platuma, miestas, kaina, rajonas =[], [],[],[],[],[],[],[],[],[],[],[],[], []
        plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas, energetineklase, ilguma, platuma, miestas, kaina, rajonas = butuduomenys(mst, i)
    
        SDB = sqlite3.connect('aruodas3.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
        Cs = SDB.cursor()

        sql = '''create table if not exists aruododuomenys
        (
        plotas integer,
        kambariuskaicius integer,
        aukstas integer,
        aukstuskaicius integer,
        metai interger,
        pastatotipas text,
        sildymas text,
        irengimas text,
        energetineklase text,
        ilguma float,
        platuma float,
        miestas text,
        kaina integer,
        rajonas text
        )
        '''
        Cs.execute(sql)

        sql_template = '''insert into aruododuomenys values (?,?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
        for a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14 in zip(plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas, energetineklase, ilguma, platuma, miestas, kaina, rajonas):
            Cs.execute(sql_template, (a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14))

        SDB.commit() # commit() būtinas, jei norite įrašyti duomenis į DB
        SDB.close() 

    

Įveskite miesto raidę ir kiek minučių veiks jūsų programa: 
 v - Vilnius, ka - Kaunas, p - Palanga, kl - Klaipėda, a - Alytus  
 Parasykite kiek minučių veiks skriptas


In [106]:
# print(len(ilguma))
# print(len(platuma))
# print(len(plotas))
# print(len(kambariusk))
# print(len(aukstas))
# print(len(aukstusk))
# print(len(metai))
# print(len(pastatotipas))
# print(len(sildymas))
# print(len(irengimas))
# print(len(energetineklase))
print(len(kaina))
print(len(miestas))

50
25


In [4]:
plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas = [], [], [], [], [], [], [], []
energetineklase = []

asas=bs1.find('dl', {'class': 'obj-details'})
# print(asas.prettify())

bandymas=asas.find_all('dt')
band=asas.find_all('dd')
bandymas=[i.text.strip() for i in bandymas]
band=[i.find('span', {'class': 'fieldValueContainer'}) for i in band]
band2= [i.text.strip() if i is not None else str(None) for i in band]
plotas.append(None)
kambariusk.append(None)
aukstas.append(None)
aukstusk.append(None)
metai.append(None)
pastatotipas.append(None)
sildymas.append(None)
irengimas.append(None)
energetineklase.append(None)

for p, k in zip(bandymas, band2):
    if p == 'Plotas:':
        plotas[-1]=k
    if p == 'Kambarių sk.:':
        kambariusk[-1]=k
    if p == 'Aukštas:':
        aukstas[-1]=k
    if p == 'Aukštų sk.:':
        aukstusk[-1]=k
    if p == 'Metai:':
        metai[-1]=k
    if p == 'Pastato tipas:':
        pastatotipas[-1]=k
    if p == 'Šildymas:':
        sildymas[-1]=k
    if p == 'Įrengimas:':
        irengimas[-1]=k
    if p == 'Pastato energijos suvartojimo klasė:':
        energetineklase[-1]=k
    print(p, k)
    
print(metai)


Namo numeris: 8
Buto numeris: 2
Plotas: 99,06 m²
Kambarių sk.: 5
Aukštas: 1
Aukštų sk.: 2
Metai: 2024
Pastato tipas: Mūrinis
Šildymas: Aeroterminis
Įrengimas: Dalinė apdaila
Pastato energijos suvartojimo klasė: A++
Ypatybės: None
Papildomos patalpos: None
Papildoma įranga: None
Apsauga: None
['2024']


In [77]:
bandymas=asas.find_all('dt')
band=asas.find_all('dd')

bandymas=[i.text for i in bandymas]
band=[i.find('span', {'class': 'fieldValueContainer'}) for i in band]
band2= [i.text if i is not None else str(None) for i in band]
for p, k in zip(bandymas, band2):
    print(p.strip(), k.strip())

Namo numeris: 34
Plotas: 63,09 m²
Kambarių sk.: 3
Aukštas: 2
Aukštų sk.: 3
Metai: 2023
Pastato tipas: Monolitinis
Šildymas: Geoterminis
Įrengimas: Dalinė apdaila
Pastato energijos suvartojimo klasė: A++
Ypatybės: None
Papildomos patalpos: None
Papildoma įranga: None
Apsauga: None


In [98]:
apci= aruodopirmas[5].find_all('img')

print(apci[0]['title'].split(',')[0])

Markučiai


In [71]:
print(aruodopirmas[5].prettify())

# apci= aruodopirmas[5].find('span', {'div': 'list-adress-v2'})
# print(apci)


<div class="advert-flex">
 <div class="list-img-v2">
  <!--
-->
  <div class="list-photo-v2">
   <a href="https://www.aruodas.lt/butai-vilniuje-markuciuose-kaukysos-g-naujai-dizainerio-irengtas-butas-projekte-1-3443992/">
    <img alt="Markučiai, Kaukysos g., 2 kambarių butas" class="impression-log-class lazyload" data-default="https://aruodas-img.dgn.lt/object_66_120337681/nuotrauka.jpg" data-extra="https://aruodas-img.dgn.lt/object_66_120792749/nuotrauka.jpg,https://aruodas-img.dgn.lt/object_66_120337723/nuotrauka.jpg,https://aruodas-img.dgn.lt/object_66_120337727/nuotrauka.jpg" data-id="3443992" data-objid="1" data-src="https://aruodas-img.dgn.lt/object_66_120337681/nuotrauka.jpg" data-srcset="https://aruodas-img.dgn.lt/object_66_120337681/nuotrauka.jpg 1x" src="https://static.aruodas.lt/static/svg/object/nophoto.svg" title="Markučiai, Kaukysos g., 2 kambarių butas | Aruodas.lt"/>
    <noscript>
     <img alt="Markučiai, Kaukysos g., 2 kambarių butas" class="impression-log-class" da